<a href="https://colab.research.google.com/github/Torikul385/NLP/blob/main/Text_Classification_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from keras import layers
import tensorflow as tf
import numpy as np

import os
import re
import string

In [2]:
!curl -O -q https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  17.3M      0  0:00:04  0:00:04 --:--:-- 17.8M


In [3]:
!ls aclImdb

imdbEr.txt  imdb.vocab	README	test  train


In [4]:
!ls aclImdb/train

labeledBow.feat  neg  pos  unsup  unsupBow.feat  urls_neg.txt  urls_pos.txt  urls_unsup.txt


In [5]:
!ls aclImdb/test

labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt


In [6]:
!rm -r aclImdb/train/unsup

In [7]:
train_ds_path = 'aclImdb/train'
test_ds_path = 'aclImdb/test'

train_raw_ds, val_raw_ds = keras.utils.text_dataset_from_directory(
    train_ds_path,
    subset='both',
    validation_split=0.2,
    seed=1111
)

test_raw_ds = keras.utils.text_dataset_from_directory(
    test_ds_path,
    seed=1111
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [8]:
text, label = next(iter(train_raw_ds))

text.shape

TensorShape([32])

In [9]:
texts = []

for text, _ in train_raw_ds.as_numpy_iterator():
    for t in text:
        t = t.decode('utf-8')
        texts.append(t)

texts[0]

'Really bad shot on video "film" made by not one, not two, but three amateur video makers.<br /><br />If you\'re going to make a bad horror film at least throw in some blood, gore and nudity. There is some blood provided by latex cut off arm props bought at a Halloween store. There are lesbians and hookers but no nudity or sex. The lesbians spend a lot of time in bed but only talking.<br /><br />There seems to be no editing effects- fades, wipes etc. Once in a while a bit of black appears to separate scenes.<br /><br />Terrible music by bad heavy metal bands whose websites take up the majority of the end credits.The werewolves are represented by rubber masks that are attached to just the "actors" face. They didn\'t even bother to apply brown makeup to their necks, arms or wrists.<br /><br />I guarantee a 10 year old with a video camera could put together a better movie.No reason at all to buy, rent or watch this film except as an example of how not to make a low budget video.'

In [10]:
VOCAB_SIZE = 20000
MAXLEN=500
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
EMBEDDING_DIM = 128


In [11]:
br = "<br />"
def customStandard(text):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, br, '')
    text = tf.strings.regex_replace(text, '[%s]'%re.escape(string.punctuation), '')
    return text

vectorizer = tf.keras.layers.TextVectorization(
    max_tokens = VOCAB_SIZE,
    standardize = customStandard,
    output_sequence_length = MAXLEN

)

vectorizer.adapt(texts)

In [12]:

def preprocess(text, labels):
    #text = tf.expand_dims(text, axis=-1)
    text = vectorizer(text)
    return text, labels

def prepare_ds(ds):
    ds = ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.prefetch(buffer_size = 10*BATCH_SIZE).cache()
    return ds

train_ds = prepare_ds(train_raw_ds)
val_ds = prepare_ds(val_raw_ds)
test_ds = prepare_ds(test_raw_ds)

In [13]:
text, labels = next(iter(train_ds))
text.shape

TensorShape([32, 500])

In [14]:
inputs = keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
model.fit(train_ds, epochs=3)

Epoch 1/3
625/625 [==============================] - 139s 216ms/step - loss: 0.5436 - accuracy: 0.6725
Epoch 2/3
625/625 [==============================] - 129s 207ms/step - loss: 0.2481 - accuracy: 0.9008
Epoch 3/3
625/625 [==============================] - 128s 205ms/step - loss: 0.1248 - accuracy: 0.9549
